In [4]:
from langchain_core.messages import HumanMessage, SystemMessage, RemoveMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.memory import InMemorySaver
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.tools.tavily_search import TavilySearchResults
import operator
from typing import Annotated, TypedDict
import uuid

from llm_models.llama3_wrapper import get_llama3_llm, get_mistral_llm
from langgraph.graph.message import add_messages

# RAG Setup: loaders, splitter, embeddings, vector store
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings


class State(TypedDict):
    question: str
    answer: str
    context: str
    summary: str
    pointers: str

# Define the logic to call the model
def call_model(state: State):
    context = state["context"]
    question = state["question"]
    pointers = state["pointers"]
    
    prompt = f"""Answer this question: {question} \nConsider these points for your analysis: {pointers} \nUse this context to answer: {context}.\nFinal Answer:"""
    
    llm = get_llama3_llm()
    answer = llm.invoke(prompt)
    
    return { "answer" : answer }

def final_answer(state: State):
    raw_output = state["answer"]
    answer = raw_output.lower().split('final answer:', 1)
    if len(answer) > 1:
        print("\n=============working===========\n")
        answer = answer[1].strip()
    else:
        answer = raw_output
    print("final_answer========", answer)
    return { "answer" : answer }


def query_decomposition_tool(state: State):
    return { "pointers" :
        f"1. Duration for which user plan to hold the investment?\n2. User's convenience and safety.\n3. Taxation.\n4. Costs and Charges"
    }

def rag_search2(state: State):
    context = '''on behalf of the Government of India. It allows investors to invest in gold in a non-physical form, making it
    a safer and more eﬃcient alternative to buying physical gold. These bonds are denominated in grams of
    gold and oﬀer an annual interest rate of 2.50 percent over and above the potential capital gains linked to the
    market price of gold.
    One of the key beneﬁts of this scheme is that it eliminates the risks associated with storing physical gold,
    such as theft or damage. Additionally, it oﬀers tax beneﬁts—especially if the bonds are held till maturity,
    as the capital gains are tax-free.
    The upcoming SGB issue under the upcoming Sovereign Gold Bond Scheme for the ﬁnancial year 2024–
    25 presents a new opportunity for investors to buy these bonds at government-declared rates. It is an
    ideal option for those looking to diversify their portfolio while beneﬁting from gold’s price appreciation

    Tradability and liquidity: SGBs are tradable on stock exchanges, making them accessible for early
    exit if required.
    Small-ticket investment: You can buy as little as one gram of gold, making them suitable for all
    budgets.
    Tax beneﬁts: Capital gains are exempt if held till maturity.
    Collateral for loans: Investors can use SGBs as security when applying for loans.
    Stay updated on the sovereign gold bond next issue date and plan your investments wisely for the next
    SGB issue date.
    Understanding the upcoming sovereign gold bond issues
    Sovereign Gold Bonds (SGBs) are a smart alternative to buying physical gold, introduced by the
    Government of India to oﬀer safer, more rewarding investment options. The sovereign gold bond scheme
    Staying informed about the upcoming SGB dates ensures that you do not miss the opportunity to invest

    Tenure: Bonds have a maturity period of 8 years, with an option for premature redemption after 5
    years.
    Interest rate: Oﬀers a ﬁxed return of 2.50% per annum, paid semi-annually.
    Issue price: Based on the average closing price of gold during the preceding week.
    Eligibility: Open to resident individuals, Hindu Undivided Families (HUFs), trusts, universities, and
    charitable institutions.
    Tradability: Bonds are listed and tradable on recognised stock exchanges.
    Minimum investment: Just 1 gram of gold, making it accessible to small investors.
    Tax beneﬁts: Capital gains on redemption are exempt from tax if held until maturity.
    Loan collateral: Bonds can be used as collateral for secured loans.
    Stay informed about the upcoming sovereign gold bond scheme 2025-26 to make the most of these
    beneﬁts.
    Next sovereign gold bond issue date: Important dates to remember'''

    return { "context" : context }

def rag_search(state: State):
    # 1) Load your PDFs
    pdf_paths = [
        "../query1_data/Gold ETF vs Physical Gold.pdf",
        "../query1_data/Sovereign Gold Bond Scheme 2025-26.pdf"
    ]

    docs = []
    for p in pdf_paths:
        loader = PyPDFLoader(p)
        # Each page comes with metadata; loader returns Document objects
        docs.extend(loader.load())

    # 2) Split into chunks (keep overlaps for context continuity)
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=900, chunk_overlap=150, separators=["\n\n", "\n", " ", ""]
    )
    splits = splitter.split_documents(docs)

    # 3) Embeddings (no API key needed)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 4) Build FAISS index
    vecstore = FAISS.from_documents(splits, embeddings)

    # 5) Create a retriever (top_k configurable inside the tool)
    retriever = vecstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

    # Define your query
    query = "What are the benefits of Sovereign Gold Bonds compared to physical gold?"

    # Search for relevant chunks
    retrieved_docs = vecstore.similarity_search(query, k=3)

    # Combine retrieved content
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    return {"context": ["context"]}

# Define the summarization function
def summarize_conversation(state: State):
    final_answer = state["answer"]
    summary = state.get("summary", "")
    if summary:
        summary_message = f"This is summary of the answer: {summary}"
    else:
        summary_prompt = f"Clean the below text properly without changing its meaning or adding additional information. Do not extend the answer and just the response as,\nSummary:\n\nText:{final_answer}"
        llm = get_llama3_llm()
        summary_message = llm.invoke(summary_prompt)
    
    return { "summary" : summary_message }

# Define the summarization function
# def summarize_conversation(state: State):
#     summary = state.get("summary", "")
#     if summary:
#         summary_message = f"This is summary of the answer: {summary}\n\nExtend the summary by taking into account the new messages above:"
#     else:
#         summary_message = "Create a summary of the conversation above:"
#     messages = [HumanMessage(content=state["question"])] + [SystemMessage(content=state["pointers"])] + [SystemMessage(content=state["context"])]
#     print("messages===================",messages)
#     llm = get_llama3_llm()
#     response = llm.invoke(messages)
#     delete_messages = [RemoveMessage(id=m.id) for m in state["context"][:-2]]
#     return {"summary": state["answer"]}


In [5]:
# Define the graph
builder = StateGraph(State)
# builder.add_node("search_web", search_web)
builder.add_node("query_decomposition_tool", query_decomposition_tool)
builder.add_node("rag_search", rag_search2)
builder.add_node("generate_answer", call_model)
builder.add_node("extract_answer", final_answer)
builder.add_node("summarize_conversation", summarize_conversation)
builder.add_edge(START, "query_decomposition_tool")
# builder.add_edge(START, "search_web")
builder.add_edge("query_decomposition_tool", "rag_search")
# builder.add_edge("search_web", "generate_answer")
# builder.add_edge("generate_answer", "rag_search")
builder.add_edge("rag_search", "generate_answer")
builder.add_edge("generate_answer", "extract_answer")
# builder.add_edge("extract_answer", "summarize_conversation")
builder.add_edge("extract_answer", END)

# Compile the graph with memory
# memory = MemorySaver()
memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

In [6]:
# if __name__ == "__main__":
    # Example question you want to test
initial_state = {
    "question": "Should I invest in gold through Sovereign Gold Bonds or buy physical gold this year?",
    "answer": "",
    "context": "",
    "summary": "",
    "pointers": "",
    "thread_id": str(uuid.uuid4())
}
result = graph.invoke(initial_state, config = {"configurable": {"thread_id": "1"}})

# Print the output (final state)
print("Final State:")
print(result)
print("\nGenerated Answer:")
print(result.get("answer"))

    # # Run the graph with the initial state
    # for event in graph.stream({"messages": [{"role": "user", "content": "Should I invest in gold through Sovereign Gold Bonds or buy physical gold this year?"}]}):
    #     for value in event.values():
    #         print("Assistant:", value["messages"][-1].content)

CUDA version: 12.1
GPU Name: NVIDIA A2


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



=============working===========

final_answer======== the final answer is sovereign gold bonds. 
explanation: considering the points mentioned in the question, investing in sovereign gold bonds seems like a better option than buying physical gold. this is because sgbs offer several benefits such as lower costs, higher returns, and greater convenience compared to traditional methods of investing in gold. they also provide tax benefits, especially if held till maturity, eliminating the risk of capital gains tax. additionally, they are easily tradable on stock exchanges, allowing for early exit if needed. furthermore, the minimum investment requirement of just 1 gram makes it accessible to small investors. overall, considering the advantages of sgbs, it appears to be a more attractive option than buying physical gold. 

note: this response is based on the information provided in the prompt and may not reflect real-time market conditions or changes in regulations. it is always recommended

In [4]:
llm2 = get_llama3_llm()

CUDA version: 12.1
GPU Name: NVIDIA A2


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.92s/it]
Device set to use cuda:0


In [5]:
normal_answer = llm2.invoke('Should I invest in gold through Sovereign Gold Bonds or buy physical gold this year?')
print(normal_answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Should I invest in gold through Sovereign Gold Bonds or buy physical gold this year? | Mint
The Reserve Bank of India (RBI) has announced the 10th tranche of the Sovereign Gold Bond (SGB) scheme, which will be open for subscription from January 3 to January 7. The SGBs are a popular investment option among Indians, especially those who want to invest in gold without buying physical bullion.
However, with the prices of gold rising over the past few years, many investors are wondering whether they should invest in gold through SGBs or buy physical gold this year. Here's a comparison between the two options:
Why invest in gold?
Gold is considered a safe-haven asset and a hedge against inflation. It also provides a diversification benefit to an investment portfolio as its price tends to move inversely to other assets like stocks and bonds.
What are Sovereign Gold Bonds (SGBs)?
SGBs are government-backed bonds that allow individuals to invest in gold by purchasing them at a fixed price. The